За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням (буде завантажено наступним кроком). Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння. Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається.

In [3]:
import pandas as pd
import zipfile
import os
from google.colab import drive

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Path to the zip file on Google Drive
zip_file_path = '/content/drive/MyDrive/Files/homework.zip'
extracted_dir_path = '/content/homework_extracted/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

In [2]:
import pandas as pd

# Function to read CSV files and add a label
def read_and_label_csv(file_path, label):
    df = pd.read_csv(file_path)
    df['label'] = label
    return df

In [6]:
# Corrected mapping of directories to labels
label_mapping = {
    'idle': 'stand',
    'walking': 'walk',
    'running': 'run',
    'stairs': 'stairs'
}

# Read all files and combine them into one dataframe
dataframes = []
for label_dir, label in label_mapping.items():
    dir_path = os.path.join(extracted_dir_path, 'data', label_dir)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        dataframes.append(read_and_label_csv(file_path, label))

# Combine all dataframes into one
combined_df = pd.concat(dataframes, ignore_index=True)

In [7]:
combined_df.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,label
0,0.272939,-0.081403,9.749189,stand
1,0.248997,-0.067038,9.777920,stand
2,0.248997,-0.076614,9.777920,stand
3,0.248997,-0.043096,9.753978,stand
4,0.272939,-0.062249,9.763555,stand


In [12]:
# Function to calculate time domain features
def calculate_time_domain_features(df, window_size=100):
    features = []
    labels = []

    for i in range(0, len(df) - window_size + 1, window_size):
        window = df.iloc[i:i + window_size]
        feature = {
            'mean_x': window['accelerometer_X'].mean(),
            'mean_y': window['accelerometer_Y'].mean(),
            'mean_z': window['accelerometer_Z'].mean(),
            'std_x': window['accelerometer_X'].std(),
            'std_y': window['accelerometer_Y'].std(),
            'std_z': window['accelerometer_Z'].std(),
            'max_x': window['accelerometer_X'].max(),
            'max_y': window['accelerometer_Y'].max(),
            'max_z': window['accelerometer_Z'].max(),
            'min_x': window['accelerometer_X'].min(),
            'min_y': window['accelerometer_Y'].min(),
            'min_z': window['accelerometer_Z'].min()
        }
        features.append(feature)
        labels.append(window['label'].mode()[0])

    features_df = pd.DataFrame(features)
    features_df['label'] = labels

    return features_df

In [13]:
# Calculate time domain features for the combined dataframe
features_df = calculate_time_domain_features(combined_df)

# Display the first few rows of the features dataframe
features_df.head()


,mean_x,mean_y,mean_z,std_x,std_y,std_z,max_x,max_y,max_z,min_x,min_y,min_z,label
0,-0.076327,0.984209,9.254356,1.358015,2.177651,1.063532,2.686294,5.995081,10.644621,-10.448297,-1.996764,4.553772,stand
1,0.121721,0.079535,9.769205,0.184411,0.128623,0.029126,0.411803,0.354342,9.945514,-0.272939,-0.138864,9.715671,stand
2,0.106925,3.475328,8.578999,0.422391,2.870480,1.116247,2.097321,7.661446,9.797073,-0.900220,-0.052672,5.995081,stand
3,0.357359,0.886429,9.495260,0.635785,2.213290,1.009558,4.118027,7.431602,11.530476,-1.570596,-0.186748,5.176265,stand
4,0.006464,2.996200,8.618647,0.410475,3.094813,1.306669,0.560243,7.876924,9.821015,-1.024718,-0.205901,4.649540,stand
